In [96]:
import pandas as pd
import numpy as np
import statsmodels as sm
import sklearn as skl
import sklearn.preprocessing as preprocessing
import sklearn.linear_model as linear_model
import sklearn.cross_validation as cross_validation
import sklearn.metrics as metrics
import sklearn.tree as tree
import seaborn as sns
import matplotlib.pylab as plt
from scipy.stats import trim_mean, kurtosis, spearmanr, kendalltau, pearsonr
from scipy.stats.mstats import mode, gmean, hmean
import plotly.plotly as py

In [168]:
data = pd.read_csv("/home/harshil/BDAP/Python Code/adultdata.csv", names = ["Age", "Workclass", "fnlwgt", 
        "Education", "Education-Num", "Martial_Status","Occupation", "Relationship", "Race", "Sex",
        "Capital_Gain", "Capital_Loss","Hour", "Country", "Target"],
        sep=r'\s*,\s*',engine='python',na_values="?")
data = data.dropna()

In [172]:
data['Target']
data['Target'] = [1 if x == '<=50K' else 0 for x in data['Target']]

In [160]:
# ----------5 number summary----------------------
data.describe()

,Age,fnlwgt,Education-Num,Capital_Gain,Capital_Loss,Hour,Target
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.0
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238,0.0
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984,0.0
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.0
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000,0.0
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.0
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000,0.0
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,0.0


In [ ]:
# ----------MEASURES OF VARIABILITY---------------
print data.std()
print data.quantile([.25,.5,.75])
print data.var()

In [ ]:
#-----------QQ PLOTS------------------------------
import scipy.stats as stats
import pylab
stats.probplot(data["Age"], dist="norm", plot=pylab)
pylab.subplot(321)
stats.probplot(data["fnlwgt"], dist="norm", plot=pylab)
pylab.subplot(322)
stats.probplot(data["Education-Num"], dist="norm", plot=pylab)
pylab.subplot(323)
stats.probplot(data["Capital_Gain"], dist="norm", plot=pylab)
pylab.subplot(324)
stats.probplot(data["Capital_Loss"], dist="norm", plot=pylab)
pylab.subplot(325)
stats.probplot(data["Hour"], dist="norm", plot=pylab)
pylab.subplot(326)

pylab.show()

In [126]:
# -----------BOX PLOT-----------------------------
# import plotly.graph_objs as go
import math as m 
fig = plt.figure(figsize=(20,15))
cols = 3
rows = m.ceil(float(data.shape[1]) / cols)
j = 0 
for i, column in enumerate(data.columns):
    if data.dtypes[column] != np.object:
        j += 1
        ax = fig.add_subplot(2, cols, j)
        ax.set_title(column)
        sns.boxplot(x = data['Target'],y = data[column])
plt.subplots_adjust(hspace=0.7, wspace=0.2)
plt.show()

In [175]:
#------------HISTOGRAM-----------------------------
# Analyse Data

import math as m 
fig = plt.figure(figsize=(20,15))
cols = 5
rows = m.ceil(float(data.shape[1]) / cols)
for i, column in enumerate(data.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if data.dtypes[column] == np.object:
        data[column].value_counts().plot(kind="bar", axes=ax)
    else:
        data[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)
plt.show()


In [176]:
# -----------SCATTER PLOT--------------------------
sns.regplot(data['Target'],data['Age'])
plt.show()
# data['Target'].head(5)

In [13]:
(data["Country"].value_counts() / data.shape[0]).head()

United-States    0.895857
Mexico           0.019748
Philippines      0.006081
Germany          0.004207
Canada           0.003716
Name: Country, dtype: float64

## OBSERVATIONS
1. Most of the data is concentrated around US

### Need to   Correlation

1. Need to convert categorical variable into numerical variables. Using Label Encoder 
2. Remove correlated variables 

In [102]:
from sklearn import preprocessing
# Encode the categorical features as numbers
def number_encode_features(df):
    result = df.copy()
    encoders = {}
#     df.apply(preprocessing.LabelEncoder().fit_transform)
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders
    
# Calculate the correlation and plot it
encoded_data, _ = number_encode_features(data)
sns.heatmap(encoded_data.corr(), square=True)
plt.show()
# Label encoder function in scikit-learn package is used  

In [177]:
# import itertools
# lis = data.select_dtypes(include=['int64'])
# correlations = {}
# columns = lis.columns.tolist()
# for col_a, col_b in itertools.combinations(columns, 2):
#     correlations[col_a + '__' + col_b] = spearmanr(lis.loc[:, col_a], lis.loc[:, col_b])
# # print correlations
# result = pd.DataFrame.from_dict(correlations, orient='index')
# result.columns = ['PCC', 'p-value']

# print(result.sort_index())


In [ ]:
data[["Education", "Education-Num"]].head(15)

- We Remove Education column as Education and Education-Num are highly correlated.
- Now to model the data, we encode the categorical variable and create histograms

In [ ]:
encoded_data, encoders = number_encode_features(data)
fig = plt.figure(figsize=(20,15))
cols = 5
rows = m.ceil(float(encoded_data.shape[1]) / cols)
for i, column in enumerate(encoded_data.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    encoded_data[column].hist(axes=ax)
    plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)
plt.show()

Before modelling, we divide the data into train and test sets. We scale all the data with mean 0 and variance 1 using StandardScalar in scikit-learn

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(encoded_data[encoded_data.columns - ["Target"]], encoded_data["Target"], train_size=0.70)
scaler = preprocessing.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train.astype('float64')), columns=X_train.columns)
X_test = scaler.transform(X_test.astype("float64"))
# ?X_train.astype

# 1 Logistic Regression

In [ ]:
cls = linear_model.LogisticRegression()

cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
sns.heatmap(cm, annot=True, fmt="d", xticklabels=encoders["Target"].classes_, yticklabels=encoders["Target"].classes_)
plt.ylabel("Real value")
plt.xlabel("Predicted value")
print "F1 score: %f" % skl.metrics.f1_score(y_test, y_pred)
coefs = pd.Series(cls.coef_[0], index=X_train.columns)
coefs.sort()
plt.subplot(2,1,2)
coefs.plot(kind="bar")
plt.show()

Using Label encoding, the marital status values are ranging from 0 to 6 and the order is important. In practice there
is no particular order in that feature. We can fix the issue using binary features by inrtoducing dummy variables.

In [ ]:
binary_data = pd.get_dummies(data)
# Let's fix the Target as it will be converted to dummy vars too
binary_data["Target"] = binary_data["Target_>50K"]
del binary_data["Target_<=50K"]
del binary_data["Target_>50K"]
plt.subplots(figsize=(20,20))
sns.heatmap(binary_data.corr(), square=True)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(binary_data[binary_data.columns - ["Target"]], binary_data["Target"], train_size=0.70)
scaler = preprocessing.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = scaler.transform(X_test)

### Logistic regression with dummy variables

In [ ]:
cls = linear_model.LogisticRegression()

cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)

plt.figure(figsize=(20,20))
plt.subplot(2,1,1)
sns.heatmap(cm, annot=True, fmt="d", xticklabels=encoders["Target"].classes_, yticklabels=encoders["Target"].classes_)
plt.ylabel("Real value")
plt.xlabel("Predicted value")
print "F1 score: %f" % skl.metrics.f1_score(y_test, y_pred)
coefs = pd.Series(cls.coef_[0], index=X_train.columns)
coefs.sort_values(inplace=True)
ax = plt.subplot(2,1,2)
coefs.plot(kind="bar")
plt.show()

# 2 Random Forest

In [113]:
# import libraries: dataframe manipulation, machine learning, os tools
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics
 # Feature Importance
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier

# change working directory to where the dataset is
# os.chdir("C:/Users/JD87417/Desktop/python work/Coursera")

# Load the dataset (http://archive.ics.uci.edu/ml/datasets/Adult)
AH_data = pd.read_csv("/media/harshil/Harshil/Ubuntu/prepdata.csv")
# AH_data = AH_data.drop('(Intercept)',axis=1)
data_clean = AH_data.dropna()

# encode categorical features
# done in R (C:\Users\JD87417\Desktop\python work\Coursera\python_adult2_clean.R)
# do one hot encoding and name the columns

# summary statistics including counts, mean, stdev, quartiles
data_clean.head(n=1)
# data_clean.dtypes # data types of each variable
# data_clean.describe()


,age,workclassFederal-gov,workclassLocal-gov,workclassNever-worked,workclassPrivate,workclassSelf-emp-inc,workclassSelf-emp-not-inc,workclassState-gov,workclassWithout-pay,fnlwgt,...,native_countryPuerto-Rico,native_countryScotland,native_countrySouth,native_countryTaiwan,native_countryThailand,income_target_50k,native_countryUnited-States,native_countryVietnam,native_countryYugoslavia,income_target>50K
0,39,0,0,0,0,0,0,1,0,77516,...,0,0,0,0,0,0,1,0,0,0


In [114]:
data_clean["Education_num"].head(1)

0    13
Name: Education_num, dtype: int64

In [115]:

# Split into training and testing sets
# Specifying predictor x variables

predictors = data_clean[["age", "workclassLocal-gov", "workclassPrivate",
"workclassSelf-emp-inc", "workclassSelf-emp-not-inc", "workclassState-gov",
"workclassWithout-pay", "fnlwgt", "education11th", "education12th",
"education1st-4th", "education5th-6th", "education7th-8th", "education9th",
"educationAssoc-acdm", "educationAssoc-voc", "educationBachelors",
"educationDoctorate", "educationHS-grad", "educationMasters",
"educationPreschool", "educationProf-school", "educationSome-college",
"Education_num", "martial_statusMarried-AF-spouse", "martial_statusMarried-civ-spouse",
"martial_statusMarried-spouse-absent", "martial_statusNever-married",
"martial_statusSeparated", "martial_statusWidowed", "occupationArmed-Forces",
"occupationCraft-repair", "occupationExec-managerial", "occupationFarming-fishing",
"occupationHandlers-cleaners", "occupationMachine-op-inspct",
"occupationOther-service", "occupationPriv-house-serv", "occupationProf-specialty",
"occupationProtective-serv", "occupationSales", "occupationTech-support",
"occupationTransport-moving", "relationshipNot-in-family", "relationshipOther-relative",
"relationshipOwn-child", "relationshipUnmarried", "relationshipWife",
"raceAsian-Pac-Islander", "raceBlack", "raceOther", "raceWhite",
"sexMale", "capital_gain", "capital_loss", "hours_per_week",
"native_countryCanada", "native_countryChina", "native_countryColumbia",
"native_countryCuba", "native_countryDominican-Republic", "native_countryEcuador",
"native_countryEl-Salvador", "native_countryEngland", "native_countryFrance",
"native_countryGermany", "native_countryGreece", "native_countryGuatemala",
"native_countryHaiti", "native_countryHoland-Netherlands", "native_countryHonduras",
"native_countryHong", "native_countryHungary", "native_countryIndia",
"native_countryIran", "native_countryIreland", "native_countryItaly",
"native_countryJamaica", "native_countryJapan", "native_countryLaos",
"native_countryMexico", "native_countryNicaragua", "native_countryOutlying-US(Guam-USVI-etc)",
"native_countryPeru", "native_countryPhilippines", "native_countryPoland",
"native_countryPortugal", "native_countryPuerto-Rico", "native_countryScotland",
"native_countrySouth", "native_countryTaiwan", "native_countryThailand",
"native_countryUnited-States",
"native_countryVietnam", "native_countryYugoslavia"]]


In [116]:
# y repsonse variable
targets = data_clean.income_target_50k

# concurrent split of x's, y, at 40%
pred_train, pred_test, tar_train, tar_test  = train_test_split(predictors, targets, test_size=.4)

# shape/dimensions of the DataFrame
print pred_train.shape
print pred_test.shape
print tar_train.shape
print tar_test.shape

(19536, 95)
(13025, 95)
(19536,)
(13025,)


In [117]:
# Build model on training data
from sklearn.ensemble import RandomForestClassifier

# n_estimators is the amount of trees to build
classifier=RandomForestClassifier(n_estimators=25)
# fit the RandomForest Model
classifier=classifier.fit(pred_train,tar_train)
# prediction scoring of the model (array of binary 0-1)
predictions=classifier.predict(pred_test)
# confusion matrix / missclassification matrix
sklearn.metrics.confusion_matrix(tar_test,predictions)
sklearn.metrics.accuracy_score(tar_test, predictions)


# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(pred_train,tar_train)
# display the relative importance of each attribute
print(model.feature_importances_)



[  8.13701024e-02   1.05498563e-02   1.29022667e-02   3.66815994e-06
   2.93246457e-02   3.22178211e-02   0.00000000e+00   1.16648142e-01
   2.56580995e-04   7.71197692e-03   3.85309926e-04   1.89189416e-02
   3.94962696e-03   3.45208204e-02   2.77934985e-03   4.47455240e-04
   1.96728446e-03   0.00000000e+00   1.85023314e-02   4.96722544e-05
   1.69174283e-04   0.00000000e+00   1.47685377e-02   1.93999244e-02
   0.00000000e+00   1.49965563e-02   8.82233042e-04   1.89814467e-02
   4.33172564e-03   8.72330854e-04   0.00000000e+00   1.54522255e-02
   8.19467784e-03   1.70296758e-05   2.94721874e-03   3.00598409e-02
   2.12815796e-02   1.28290498e-03   1.71722712e-03   5.40193330e-03
   2.21521200e-04   2.29288657e-02   2.09944095e-02   1.18919942e-02
   3.48963417e-03   1.67327868e-03   1.45854663e-02   6.85665533e-03
   3.99312044e-03   4.28181492e-02   9.29087839e-03   3.00898706e-02
   1.53590874e-02   4.13465264e-03   6.55248397e-02   7.89191175e-02
   1.84628719e-04   1.10838363e-03

In [118]:
print(max(model.feature_importances_))
max_val = np.where(model.feature_importances_ == max(model.feature_importances_))

min_val = np.where(model.feature_importances_ == min(model.feature_importances_))



0.116648142339


In [119]:
print(max_val, min_val)

((array([7]),), (array([ 6, 17, 21, 24, 30, 59, 61, 62, 63, 64, 66, 67, 69, 70, 72, 74, 75,
       76, 79, 82, 83, 85, 86, 88, 90, 91, 94]),))


In [120]:
"""
Running a different number of trees and see the effect
 of that on the accuracy of the prediction
"""

trees=range(25)
accuracy=np.zeros(25)

for idx in range(len(trees)):
    classifier=RandomForestClassifier(n_estimators=idx + 1)
    classifier=classifier.fit(pred_train,tar_train)
    predictions=classifier.predict(pred_test)
    accuracy[idx]=sklearn.metrics.accuracy_score(tar_test, predictions)

plt.cla()
plt.plot(trees, accuracy)
plt.show()